In [1]:
import pandas as pd
import yfinance as yf
import datetime

# display entire table
pd.set_option('display.max_rows', None)

In [2]:
start, end = "2021-07-01", "2021-08-05"
ticker = 'aapl'

#### Generate dummy reference data
Errors and changes added to reference_df

In [3]:
n, m = 18, 8
date_range = pd.date_range(start=start, end=end)
currency = ['USD'] * (len(date_range) - n) + ['EUR'] * n
industry = ['Technology'] * (len(date_range) - m) + ['Tech'] * 5 + ['Technology'] * (m-5)
region = ['EMEA'] * (len(date_range) - 2) + ['APAC'] *2 

d = {'day': date_range, 'currency': currency, 'industry': industry, 'region': region}
reference_df = pd.DataFrame(data=d)
reference_df.loc[8:10, ['currency', 'industry', 'region']] = ['N/A', 'N/A', 'N/A']
reference_df.set_index('day', inplace=True)
reference_df

,currency,industry,region
day,,,
2021-07-01,USD,Technology,EMEA
2021-07-02,USD,Technology,EMEA
2021-07-03,USD,Technology,EMEA
2021-07-04,USD,Technology,EMEA
2021-07-05,USD,Technology,EMEA
2021-07-06,USD,Technology,EMEA
2021-07-07,USD,Technology,EMEA
2021-07-08,USD,Technology,EMEA
2021-07-09,N/A,N/A,N/A


#### Get historical stock prices (variable=ticker)
stock_price

In [4]:
stock_price = yf.Ticker(ticker).history(start=start, end=end)
stock_price.index = pd.to_datetime(stock_price.index)
stock_price = stock_price.iloc[:, :5]
stock_price

,Open,High,Low,Close,Volume
Date,,,,,
2021-06-30,135.966285,137.204435,135.666731,136.755112,63261400
2021-07-01,136.395656,137.124559,135.556901,137.064651,52485800
2021-07-02,137.693695,139.790559,137.543925,139.750626,78852600
2021-07-06,139.860461,142.935840,139.860461,141.807541,108181800
2021-07-07,143.325256,144.673243,142.446583,144.353729,104911600
2021-07-08,141.368191,143.844476,140.459549,143.025711,105575500
2021-07-09,142.536444,145.432099,142.436587,144.892914,99890800
2021-07-12,145.991277,146.101113,143.784577,144.283829,76299700
2021-07-13,143.814527,147.239404,143.415132,145.422119,100827100


#### Drop non trading days from the reference_df

In [5]:
reference_df = reference_df[reference_df.index.isin(stock_price.index)]
reference_df

,currency,industry,region
day,,,
2021-07-01,USD,Technology,EMEA
2021-07-02,USD,Technology,EMEA
2021-07-06,USD,Technology,EMEA
2021-07-07,USD,Technology,EMEA
2021-07-08,USD,Technology,EMEA
2021-07-09,N/A,N/A,N/A
2021-07-12,USD,Technology,EMEA
2021-07-13,USD,Technology,EMEA
2021-07-14,USD,Technology,EMEA


#### Capture changes in reference_df

In [7]:
ref_df_unpivoted = reference_df.reset_index().melt(id_vars='day').set_index(['day', 'variable'])
ref_df_unpivoted.sort_index(inplace=True)
ref_df_unpivoted.tail(12)

value
day        variable            
2021-07-30 currency         EUR
           industry        Tech
           region          EMEA
2021-08-02 currency         EUR
           industry        Tech
           region          EMEA
2021-08-03 currency         EUR
           industry  Technology
           region          EMEA
2021-08-04 currency         EUR
           industry  Technology
           region          APAC

In [8]:
from pprint import pprint
change = []
for i in range(len(reference_df.index)-1):
    for j in range(len(reference_df.columns)):
        chg = (reference_df.iloc[[i]].index[0], reference_df.columns[j], reference_df.iloc[i, j]!=reference_df.iloc[i+1, j])
        change.append(chg)

# List of tuples - similar to an unpivoted structure-
# within each tuple the 1st item: day, 2nd: field, 3rd: True/False indicating changes
pprint(change[10:20])

changes_df = pd.DataFrame(change, columns= ['day', 'variable', 'changed']).set_index(['day', 'variable'])

# keep only the changes
changes_df = changes_df[changes_df['changed']==True].drop(['changed'], axis=1)

# populate changes_df with data
changes_df = changes_df.merge(ref_df_unpivoted, left_index=True, right_index=True)
changes_df = changes_df.sort_index()


display(changes_df.iloc[:12,:])

[(Timestamp('2021-07-07 00:00:00'), 'industry', False),
 (Timestamp('2021-07-07 00:00:00'), 'region', False),
 (Timestamp('2021-07-08 00:00:00'), 'currency', True),
 (Timestamp('2021-07-08 00:00:00'), 'industry', True),
 (Timestamp('2021-07-08 00:00:00'), 'region', True),
 (Timestamp('2021-07-09 00:00:00'), 'currency', True),
 (Timestamp('2021-07-09 00:00:00'), 'industry', True),
 (Timestamp('2021-07-09 00:00:00'), 'region', True),
 (Timestamp('2021-07-12 00:00:00'), 'currency', False),
 (Timestamp('2021-07-12 00:00:00'), 'industry', False)]


value
day        variable            
2021-07-08 currency         USD
           industry  Technology
           region          EMEA
2021-07-09 currency         N/A
           industry         N/A
           region           N/A
2021-07-16 currency         USD
2021-07-28 industry  Technology
2021-08-02 industry        Tech
2021-08-03 region          EMEA

In [9]:
forever = pd.to_datetime('2222-12-31')

reference_df_start = reference_df.iloc[[0]].reset_index()
reference_df_start.rename(columns={'day': 'valid_from'}, inplace=True)
reference_df_start['valid_from'] = reference_df_start['valid_from'] # + pd.Timedelta(seconds=1)
# adding 1 day minus a second

#every time where there is a change, the valid_to changes to "forever - 1 second" and valid_from to the day of a change
reference_df_start['valid_to'] = forever - pd.Timedelta(seconds=1)

reference_df_start = reference_df_start[['valid_from', 'valid_to', 'currency', 'industry', 'region']]
reference_df_start.set_index('valid_from', inplace=True)
reference_df_start

,valid_to,currency,industry,region
valid_from,,,,
2021-07-01,2222-12-30 23:59:59,USD,Technology,EMEA


In [10]:
# adding changes
changes_pivot_df = changes_df.reset_index()
changes_pivot_df = changes_pivot_df.pivot(index='day', columns='variable', values='value')
changes_pivot_df

variable,currency,industry,region
day,,,
2021-07-08,USD,Technology,EMEA
2021-07-09,N/A,N/A,N/A
2021-07-16,USD,NaN,NaN
2021-07-28,NaN,Technology,NaN
2021-08-02,NaN,Tech,NaN
2021-08-03,NaN,NaN,EMEA


In [11]:
df = pd.concat([reference_df_start, changes_pivot_df])
df.index.rename('valid_from', inplace=True)
df = df.reset_index()
display(df)
# df.reset_index()

,valid_from,valid_to,currency,industry,region
0,2021-07-01,2222-12-30 23:59:59,USD,Technology,EMEA
1,2021-07-08,NaT,USD,Technology,EMEA
2,2021-07-09,NaT,N/A,N/A,N/A
3,2021-07-16,NaT,USD,NaN,NaN
4,2021-07-28,NaT,NaN,Technology,NaN
5,2021-08-02,NaT,NaN,Tech,NaN
6,2021-08-03,NaT,NaN,NaN,EMEA


In [12]:
for row in range(df.shape[0]-1):
    valid_to = df.iloc[row + 1 , 0] # - pd.Timedelta(seconds=1)
    df.loc[row, 'valid_to'] = valid_to
    
# the last value should have no expiry -> valid to = forever
df.iloc[-1, 1] =  forever
display(df)

,valid_from,valid_to,currency,industry,region
0,2021-07-01,2021-07-08,USD,Technology,EMEA
1,2021-07-08,2021-07-09,USD,Technology,EMEA
2,2021-07-09,2021-07-16,N/A,N/A,N/A
3,2021-07-16,2021-07-28,USD,NaN,NaN
4,2021-07-28,2021-08-02,NaN,Technology,NaN
5,2021-08-02,2021-08-03,NaN,Tech,NaN
6,2021-08-03,2222-12-31,NaN,NaN,EMEA
